### OpenCV的简单旋转问题

图像在旋转时会被“剪切”，整个图像不会保留在视场中。

In [138]:
import cv2

image = cv2.imread('pill_01.png')
(h, w, d) = image.shape

center = (w // 2, h // 2)

# 以图片中心点为转轴，顺时针旋转45度，放缩比例为1.0，得到旋转矩阵
M = cv2.getRotationMatrix2D(center, -45, 1.0)

# 将原图片像素坐标按照旋转矩阵做仿射运算，得到旋转后的像素坐标，并指定返回图片显示范围 [0 ~ w, 0 ~ h]
rotated = cv2.warpAffine(image, M, (w, h))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey(0)

-1

### 实施不会切断图像的旋转功能

In [158]:
import numpy as np

def getNewRotationMatrix2D(w, h, center, angle, scale):
    M = cv2.getRotationMatrix2D(center, angle, scale)
    points = np.array([[0, 0], [w, 0], [0, h], [w, h]]).T
    nPoints = np.matmul(M[:, :2], points) + M[:, 2:]
    minX = np.min(nPoints[0, :])
    maxX = np.max(nPoints[0, :])
    minY = np.min(nPoints[1, :])
    maxY = np.max(nPoints[1, :])
    M[0, 2] -= minX
    M[1, 2] -= minY
    return M, (int(maxX - minX), int(maxY - minY))

In [165]:
(h, w, d) = image.shape
center = (w // 2, h // 2)

M, (nW, nH) = getNewRotationMatrix2D(w, h, center, -45, 1.0)
rotated = cv2.warpAffine(image, M, (nW, nH))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey(0)

-1

转换为灰度，使其模糊，并应用边缘检测以显示药丸的轮廓：

In [173]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(gray, 20, 100)

cv2.imshow("edged", edged)
cv2.waitKey(0)

-1

应用轮廓检测来找到药丸的轮廓：

In [174]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

从图像中提取药丸的ROI：

In [186]:
# 确保至少找到一个轮廓
if len(cnts[0]) > 0:
    # 抓住最大轮廓，然后为药丸画一个面具
    c = max(cnts[0], key=cv2.contourArea)
    mask = np.zeros(gray.shape, dtype="uint8")
    cv2.drawContours(mask, [c], -1, 255, -1)
    
    # 计算药丸的边界框，然后提取ROI，并用面具遮盖
    (x, y, w, h) = cv2.boundingRect(c)
    imageROI = image[y:y + h, x:x + w]
    maskROI = mask[y:y + h, x:x + w]
    imageROI = cv2.bitwise_and(imageROI, imageROI, mask=maskROI)

    cv2.imshow("maskROI", maskROI)
    cv2.waitKey(0)
    cv2.imshow("imageROI", imageROI)
    cv2.waitKey(0)

-1

使用两种旋转方式，分别进行360度周转：

In [184]:
for angle in np.arange(0, 360, 45):
    (h, w, d) = imageROI.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(imageROI, M, (w, h))
    
    cv2.imshow("Rotated (Problematic)", rotated)
    cv2.waitKey(0)

In [188]:
for angle in np.arange(0, 360, 45):
    (h, w, d) = imageROI.shape
    center = (w // 2, h // 2)
    M, (nW, nH) = getNewRotationMatrix2D(w, h, center, angle, 1.0)
    rotated = cv2.warpAffine(imageROI, M, (nW, nH))
    
    cv2.imshow("Rotated (Correct)", rotated)
    cv2.waitKey(0)